Tomi Salomaa <br>
student number <br>
  <br>
January, 31, 2021  <br>

# Exercise 1 | TKO_2096 Application of Data Analysis 2021

#### Nested cross-validation for K-nearest neighbors <br>
- Use Python 3 to program a nested cross-validation for the k-nearest neighbors (kNN) method so that the number of neighbours k is automatically selected from the range 1 to 10. In other words, the base learning algorithm is kNN but the actual learning algorithm, whose prediction performance will be evaluated with nested CV, is kNN with automatic CV-based model selection (see the lectures and the pseudo codes presented on them for more info on this interpretation).
- As a kNN implementation, you can use sklearn: http://scikit-learn.org/stable/modules/neighbors.html but your own kNN implementation can also be used if you like to keep more control on what is happening in the learning process. The CV implementation should be easily modifiable, since the forthcoming exercises involve different problem-dependent CV variations.
- Use the nested CV implementation on the iris data and report the resulting classification accuracy. Hint: you can use the nested CV example provided on sklearn documentation: https://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html as a starting point and compare your nested CV implementation with that but do NOT use the ready made CV implementations of sklearn as the idea of the exercise is to learn to split the data on your own. The other exercises need more sophisticated data splitting which are not necessarily available in libraries.
- Return your solution for each exercise BOTH as a Jupyter Notebook file and as a PDF-file made from it.
- Return the report to the course page on **Monday 1st of February** at the latest.  

## Import libraries

In [43]:
#In this cell import all libraries you need. For example: 
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, metrics
from sklearn.neighbors import KNeighborsClassifier
from random import randrange, seed

## Methods for the complete approach below

In [44]:
# Load iris dataset and separate to features and (numerical) labels
# Form a collective dataset to use later
irisData = datasets.load_iris()
X_iris = irisData.data
y_iris = irisData.target
iris_collective = [None]*len(X_iris)

for i in range (0,len(X_iris)):
  iris_collective[i] = np.append(X_iris[i], y_iris[i])

In [45]:
# Just a check to see the arrays are correct
print(X_iris[0])
print(y_iris[0])
print(iris_collective[0])

[5.1 3.5 1.4 0.2]
0
[5.1 3.5 1.4 0.2 0. ]


In [46]:
# Cross validation split method
# !!! DOES NOT stratify, only pops randomly !!!
#
# Defaults the amount of folds to 10 if not spesicied otherwise.
# Defines fold element size by dividing the parameter dataset length
# with parameter folds and returns the resulting split as a list of arrays.
def cross_validation_split(dataset, folds=10):
  dataset_split = list()
  dataset_copy = list(dataset)
  fold_size = int(len(dataset) / folds)
  for i in range(folds):
    fold = list()
    while len(fold) < fold_size:
      index = randrange(len(dataset_copy))
      fold.append(dataset_copy.pop(index))
    dataset_split.append(fold)
  return dataset_split

In [47]:
# Inner cross validation loop.
#
# Inspired by https://towardsdatascience.com/build-knn-from-scratch-python-7b714c47631a
# and modded to work with the numpy array setup used in this excercise.
#
# Splits the given dataset by using cross_validation_split method with
# user defined parameters.
# Creates a scikit knn classifier with neighbours of n 1-10,
# fits and saves the mean accuracy score of each classifier,
# after which returns the best observed neighbour amount
# based on the mean scores of the classifiers.
def cross_validation_inner(dataset, folds):
  folds = cross_validation_split(dataset, folds)
  mean_scores = []
  absolute_scores = []
  result = []
  
  # perform loop for each k value between [1-10]
  for i in range (1,11):
    scores = []
    knn = KNeighborsClassifier(n_neighbors=i)
    
    # For given k value between 1-10, go through splitting to train and valuate.
    # The construct of outer method is used here so valuate is still named as
    # test_set.
    for fold in range (len(folds)):
      train_set = list(folds)
      train_set.pop(fold)
      train_set = sum(train_set, [])
      test_set = list()

      for arr in folds[fold]:
        arr_copy = list(arr)
        test_set.append(arr_copy)

      # Building y_ and x_ train/test arrays and fitting the knn classifier.
      y_train = [arr[-1] for arr in train_set]
      x_train = [train[:-1] for train in train_set]
      knn.fit(x_train, y_train)
      y_test = [arr[-1] for arr in test_set]
      x_test = [test[:-1] for test in test_set]

      # Scoring by accuracy
      acc = knn.score(x_test, y_test)
      scores.append(acc)
    
    absolute_scores.append(scores)
    mean_scores.append(np.mean(scores))
  
  # Checks which index has the highest mean score.
  # Since the k values are tested in order, the index value can also
  # be used to define the k value.
  # For information purposes additional data is returned as results also.
  result.append(np.argmax(mean_scores)+1)
  result.append(absolute_scores)
  result.append(mean_scores)

  return result

In [57]:
# Nested cross validation (or the outer loop).
# Splits the given dataset and feeds each fold to the inner loop
# to receive the best observed neighbour amount. Uses this value
# for the scikit knn classifier.
#
# This method is in many ways similar to the inner method. Look for
# cross_validation_inner() above for further comments.
def cross_validation_nested(dataset, outer_folds=10, inner_folds=5):
  folds = cross_validation_split(dataset, outer_folds)
  scores = []
  neighbours = []
  inner_scores_absolute = []
  inner_scores_mean = []
  
  for fold in range (len(folds)):
    train_set = list(folds)
    train_set.pop(fold)
    train_set = sum(train_set, [])
    test_set = list()

    for arr in folds[fold]:
      arr_copy = list(arr)
      test_set.append(arr_copy)

    # Set the observed best neighbour k value as a parameter for knn and save
    # other inner results for later presenting.
    results_from_inner = cross_validation_inner(train_set, inner_folds)
    neighbours.append(results_from_inner[0])
    inner_scores_absolute.append(results_from_inner[1])
    inner_scores_mean.append(results_from_inner[2])
    knn = KNeighborsClassifier(n_neighbors = results_from_inner[0])
    
    y_train = [arr[-1] for arr in train_set]
    x_train = [train[:-1] for train in train_set]
    knn.fit(x_train, y_train)
    y_test = [arr[-1] for arr in test_set]
    x_test = [test[:-1] for test in test_set]
    acc = knn.score(x_test, y_test)
    scores.append(acc)
  
  present_mean_scores = np.mean(scores)
  present_scores = np.around(scores, decimals = 5)

  print('-'*42)
  print('OVERALL MEAN ACCURACY:',present_mean_scores)
  print('-'*42)
  print('\nFold spesific info for',outer_folds,'outer folds and',inner_folds,
        'inner folds.')
  print('*'*60)
  for i in range (0,len(scores)):
    print('Fold',i+1,'>>','Score:',present_scores[i],'with k =',neighbours[i])
    for j in range (0,len(inner_scores_absolute[i])):
      print('   Inner fold',j+1,'>>','Absolute scores:',inner_scores_absolute[i][j])
      print('   Inner fold',j+1,'>>','Mean score:',inner_scores_mean[i][j])

In [61]:
#In this cell run your script for nested CV and print the result.
cross_validation_nested(iris_collective, 5, 10)

------------------------------------------
OVERALL MEAN ACCURACY: 0.9533333333333334
------------------------------------------

Fold spesific info for 5 outer folds and 10 inner folds.
************************************************************
Fold 1 >> Score: 0.96667 with k = 6
   Inner fold 1 >> Absolute scores: [1.0, 1.0, 1.0, 0.9166666666666666, 1.0, 1.0, 0.8333333333333334, 1.0, 1.0, 0.9166666666666666]
   Inner fold 1 >> Mean score: 0.9666666666666666
   Inner fold 2 >> Absolute scores: [1.0, 0.9166666666666666, 1.0, 0.9166666666666666, 1.0, 1.0, 0.8333333333333334, 1.0, 1.0, 0.9166666666666666]
   Inner fold 2 >> Mean score: 0.9583333333333333
   Inner fold 3 >> Absolute scores: [1.0, 1.0, 1.0, 0.9166666666666666, 1.0, 1.0, 0.8333333333333334, 1.0, 1.0, 1.0]
   Inner fold 3 >> Mean score: 0.975
   Inner fold 4 >> Absolute scores: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8333333333333334, 1.0, 1.0, 1.0]
   Inner fold 4 >> Mean score: 0.9833333333333334
   Inner fold 5 >> Absolute scor

<h3>Thoughts and analysis of the task:</h3>
<p>
With a busy couple of weeks a decision to use scikit's available KNN-classifier was made to reduce potential errors present from a self-made original KNN-algorithm. Creating such an algorithm would be an interesting challenge, though and will most likely be done later.
</p>
<p>
The concrete outcome of this task were the two cross validation methods which when used together formed a nested cross validation approach to choosing and evaluating the optimal parameter value for KNN neighbors. Along these two methods a third method was created for splitting the data to be used for cross validation.
</p>
<p>
Some problems / thoughts for the general functionality of the proposed solution rose up during the programming process:
<ul>
<li>Currently the use of KNN classifier is deeply integrated within the cross validation solution. For the task at hand this works but in general the CV approach should be model agnostic and just execute the validation.</li>
<li>Splitting of the dataset to be used should be possible with stratifying. Currently randomness is used to perform the split. With a few test runs this seemed to work ok but it is reasonable to note that the iris dataset has 1:1:1 relation with target classes; random split probably won't do if the dataset is imbalanced target-wise.</li>
<li>The inner CV performs folds the same way as the outer (though fold amounts can be specified through parameters). LOOCV was thought of as a possibility for inner CV approach and probably this should be coded as an option to the proposed solution.</li>
</ul>
</p>
<p>
Overall the nested cross validation seemed to give out mean values that were a bit on the high end. If this is due to poor splitting / handling of the data I am uncertain as similar results seemed to be reported with Iris dataset from what could be gathered on some outside sources (so perhaps the approach itself with nested CV and KNN just works well with the dataset). Though, it is highly possible that the code makes overfitting possible / handles train & test datasets incorrectly. To be honest, much of the brain resources were used to learning Python while attempting to grasp the solution as well, so something may have slipped between fingers.
</p>

<h3>Used outside references and sources beside lectures:</h3>
<p>
https://towardsdatascience.com/build-knn-from-scratch-python-7b714c47631a<br/>
https://towardsdatascience.com/implementing-k-nearest-neighbors-with-scikit-learn-9e4858e231ea<br/>
https://towardsdatascience.com/complete-guide-to-pythons-cross-validation-with-examples-a9676b5cac12<br/>
https://machinelearningmastery.com/nested-cross-validation-for-machine-learning-with-python/<br/>
https://machinelearningmastery.com/implement-resampling-methods-scratch-python/
</p>